In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install facenet-pytorch timm albumentations --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade --force-reinstall Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 45.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires Pillow<10.3.0,>=10.2.0, but you have pillow 11.2.1 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [3]:
import os
import cv2
import numpy as np
import torch
import timm
import torch.nn as nn
from tqdm import tqdm
from PIL import Image
from facenet_pytorch import MTCNN
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, roc_auc_score


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)


Using device: cuda


In [5]:
REAL_PATH = "/kaggle/input/original-ds/Original"
FAKE_PATH = "/kaggle/input/fake-800/Fake800"
NUM_FRAMES_PER_VIDEO = 5


In [7]:
mtcnn = MTCNN(image_size=224, margin=0, device=device)

def extract_faces(video_dir, label, num_frames=5):
    face_list = []
    label_list = []
    
    for video_file in tqdm(os.listdir(video_dir), desc=f"Processing {label} videos"):
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_idxs = np.linspace(0, total_frames-1, num_frames, dtype=int)

        for idx in frame_idxs:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            success, frame = cap.read()
            if not success:
                continue

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face = mtcnn(rgb)

            if face is not None:
                face_list.append(face)
                label_list.append(label)
        cap.release()

    return face_list, label_list


In [9]:
real_faces, real_labels = extract_faces(REAL_PATH, 0, NUM_FRAMES_PER_VIDEO)
fake_faces, fake_labels = extract_faces(FAKE_PATH, 1, NUM_FRAMES_PER_VIDEO)

X_faces = real_faces + fake_faces
y_labels = real_labels + fake_labels


Processing 1 videos: 100%|██████████| 804/804 [33:27<00:00,  2.50s/it]


In [10]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm

In [11]:
def prepare_rgb(face_tensor):
    return face_tensor  # already in [3, 224, 224], normalized to [0, 1]

def prepare_fft(image_tensor):
    image_np = image_tensor.numpy().transpose(1, 2, 0)
    gray = cv2.cvtColor((image_np * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    magnitude_spectrum = cv2.resize(magnitude_spectrum, (image_np.shape[1], image_np.shape[0]))
    magnitude_spectrum = (magnitude_spectrum - magnitude_spectrum.min()) / (magnitude_spectrum.max() - magnitude_spectrum.min() + 1e-8)
    magnitude_tensor = torch.tensor(magnitude_spectrum, dtype=torch.float32).unsqueeze(0)
    return magnitude_tensor

def prepare_motion(current_tensor, previous_tensor):
    motion = current_tensor - previous_tensor
    motion = torch.abs(motion)
    return motion

In [12]:
from torch.utils.data import Dataset

class MultiStreamDeepfakeDataset(Dataset):
    def __init__(self, rgb_faces, fft_images, motion_images, labels, transform=None):
        self.rgb_faces = rgb_faces
        self.fft_images = fft_images
        self.motion_images = motion_images
        self.labels = labels
        self.transform = transform  # Optional

    def __len__(self):
        return len(self.rgb_faces)

    def __getitem__(self, idx):
        rgb = self.rgb_faces[idx]
        fft = self.fft_images[idx]
        motion = self.motion_images[idx]
        label = self.labels[idx]

        # Apply transforms if needed
        if self.transform:
            rgb = self.transform(rgb)
            fft = self.transform(fft)
            motion = self.transform(motion)

        return rgb, fft, motion, torch.tensor(label, dtype=torch.float32)



In [13]:
import torch.nn.functional as F

X_faces_tensor = [
    F.interpolate(face.float().unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
    for face in X_faces
]



In [14]:
class CBAM(nn.Module):
    def __init__(self, channels, reduction_ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction_ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(channels // reduction_ratio, channels, 1, bias=False),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        ca = self.channel_attention(x)
        x = x * ca
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        sa = self.spatial_attention(torch.cat([avg_out, max_out], dim=1))
        x = x * sa
        return x

In [15]:
class MultiStreamEfficientNet(nn.Module):
    def __init__(self):
        super(MultiStreamEfficientNet, self).__init__()
        self.rgb_model = models.efficientnet_b0(pretrained=True).features
        self.fft_model = models.efficientnet_b0(pretrained=True).features
        self.motion_model = models.efficientnet_b0(pretrained=True).features

        self.rgb_attention = CBAM(1280)
        self.fft_attention = CBAM(1280)
        self.motion_attention = CBAM(1280)

        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=1280, nhead=8, batch_first=True),
            num_layers=2
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(1280 * 3, 1),
            nn.Sigmoid()
        )

    def forward(self, rgb, fft, motion):
        rgb_feat = self.rgb_model(rgb)
        fft_feat = self.fft_model(fft)
        motion_feat = self.motion_model(motion)

        rgb_feat = self.rgb_attention(rgb_feat)
        fft_feat = self.fft_attention(fft_feat)
        motion_feat = self.motion_attention(motion_feat)

        b, c, h, w = rgb_feat.size()
        rgb_feat = rgb_feat.view(b, c, -1).permute(0, 2, 1)
        fft_feat = fft_feat.view(b, c, -1).permute(0, 2, 1)
        motion_feat = motion_feat.view(b, c, -1).permute(0, 2, 1)

        combined = torch.cat([rgb_feat, fft_feat, motion_feat], dim=1)
        fused = self.transformer(combined)
        fused = fused.permute(0, 2, 1).view(b, 1280 * 3, h, w)

        output = self.classifier(fused)
        return output.squeeze(1)

In [16]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch in tqdm(dataloader, desc="Training"):
        rgb = batch['rgb'].to(device)
        fft = batch['fft'].to(device)
        motion = batch['motion'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(rgb, fft, motion)
        loss = criterion(outputs.unsqueeze(1), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def validate(model, dataloader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            rgb = batch['rgb'].to(device)
            fft = batch['fft'].to(device)
            motion = batch['motion'].to(device)
            labels = batch['label'].to(device)

            outputs = model(rgb, fft, motion)
            preds.extend(outputs.cpu().numpy().squeeze().tolist())
            targets.extend(labels.cpu().numpy().squeeze().tolist())

    preds = np.array(preds) > 0.5
    accuracy = accuracy_score(targets, preds)
    precision = precision_score(targets, preds)
    recall = recall_score(targets, preds)
    f1 = f1_score(targets, preds)
    auc = roc_auc_score(targets, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "auc": auc}

In [17]:
# Utility functions
def prepare_fft(tensor):
    gray = tensor.mean(dim=0, keepdim=True)
    fft = torch.fft.fft2(gray)
    fft_mag = torch.abs(fft)
    fft_norm = (fft_mag - fft_mag.min()) / (fft_mag.max() - fft_mag.min() + 1e-8)
    fft_img = fft_norm.expand(3, -1, -1)
    return fft_img

def prepare_motion(current_tensor, prev_tensor):
    return torch.abs(current_tensor - prev_tensor)

# Input: X_faces_tensor (list of torch tensors) and y_labels (list of 0/1)
fft_tensors = [prepare_fft(face) for face in X_faces_tensor]

motion_tensors = [
    prepare_motion(X_faces_tensor[i], X_faces_tensor[i-1]) if i > 0 else torch.zeros_like(X_faces_tensor[i])
    for i in range(len(X_faces_tensor))
]

# Create dataset
dataset = MultiStreamDeepfakeDataset(X_faces_tensor, fft_tensors, motion_tensors, y_labels)


In [18]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

train_set, val_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)


In [19]:
import torch
import torch.nn as nn
from torchvision import models

# CBAM Attention Block (simplified)
class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAM, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel attention
        ca = self.channel_attention(x)
        x = x * ca

        # Spatial attention
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        sa = torch.cat([avg_out, max_out], dim=1)
        sa = self.spatial_attention(sa)
        x = x * sa
        return x

# Stream module (EfficientNet + CBAM)
class StreamNet(nn.Module):
    def __init__(self):
        super(StreamNet, self).__init__()
        base_model = models.efficientnet_b0(pretrained=True)
        self.features = base_model.features
        self.cbam = CBAM(1280)
        self.pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.features(x)
        x = self.cbam(x)
        x = self.pool(x).view(x.size(0), -1)  # Flatten
        return x

# MultiStream Model
class MultiStreamModel(nn.Module):
    def __init__(self):
        super(MultiStreamModel, self).__init__()
        self.rgb_stream = StreamNet()
        self.fft_stream = StreamNet()
        self.motion_stream = StreamNet()

        self.classifier = nn.Sequential(
            nn.Linear(1280 * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, rgb, fft, motion):
        rgb_feat = self.rgb_stream(rgb)
        fft_feat = self.fft_stream(fft)
        motion_feat = self.motion_stream(motion)

        combined = torch.cat([rgb_feat, fft_feat, motion_feat], dim=1)
        return self.classifier(combined)


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiStreamModel().to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 119MB/s] 


In [21]:
!pip install tqdm


In [23]:
from tqdm import tqdm


In [24]:
from tqdm import tqdm

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)

    for rgb, fft, motion, labels in train_loader_tqdm:
        rgb, fft, motion, labels = rgb.to(device), fft.to(device), motion.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(rgb, fft, motion).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_loader_tqdm.set_postfix(loss=loss.item())

    model.eval()
    val_loss = 0.0
    correct, total = 0, 0

    val_loader_tqdm = tqdm(val_loader, desc="Validating", leave=False)

    with torch.no_grad():
        for rgb, fft, motion, labels in val_loader_tqdm:
            rgb, fft, motion, labels = rgb.to(device), fft.to(device), motion.to(device), labels.to(device).float()

            outputs = model(rgb, fft, motion).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = (outputs > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)

    acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {acc:.4f}")



Epoch [1/10]


Epoch [1/10], Train Loss: 85.9048, Val Loss: 13.2706, Val Acc: 0.9200

Epoch [2/10]


Epoch [2/10], Train Loss: 30.1417, Val Loss: 12.0706, Val Acc: 0.9396

Epoch [3/10]


Epoch [3/10], Train Loss: 20.7694, Val Loss: 13.1820, Val Acc: 0.9422

Epoch [4/10]


Epoch [4/10], Train Loss: 12.5793, Val Loss: 11.4029, Val Acc: 0.9538

Epoch [5/10]


...

Epoch [6/10], Train Loss: 8.6310, Val Loss: 16.6622, Val Acc: 0.9387

Epoch [7/10]


Epoch [7/10], Train Loss: 6.9385, Val Loss: 13.2371, Val Acc: 0.9556

Epoch [8/10]


Epoch [8/10], Train Loss: 6.9509, Val Loss: 16.0349, Val Acc: 0.9484

Epoch [9/10]


Epoch [9/10], Train Loss: 8.5136, Val Loss: 10.7664, Val Acc: 0.9582

Epoch [10/10]


Epoch [10/10], Train Loss: 4.2649, Val Loss: 13.3352, Val Acc: 0.9547


In [56]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

all_preds, all_labels = [], []

model.eval()
with torch.no_grad():
    for rgb, fft, motion, labels in val_loader:
        rgb, fft, motion = rgb.to(device), fft.to(device), motion.to(device)
        labels = labels.to(device).long()

        outputs = model(rgb, fft, motion).squeeze().cpu()
        preds = (outputs > 0.5).long()
        

        all_preds.extend(preds.tolist())
        all_labels.extend(labels.cpu().tolist())

# Classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Real", "Fake"]))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

# AUC-ROC Score
prob_outputs = [float(o) for o in outputs]  # Last batch only, optional improvement below
roc_auc = roc_auc_score(all_labels, all_preds)
print("ROC AUC Score:", roc_auc)


Classification Report:
              precision    recall  f1-score   support

        Real       0.96      0.90      0.93       370
        Fake       0.95      0.98      0.97       755

    accuracy                           0.95      1125
   macro avg       0.96      0.94      0.95      1125
weighted avg       0.95      0.95      0.95      1125

Confusion Matrix:
 [[334  36]
 [ 15 740]]
ROC AUC Score: 0.9414175765169143


In [25]:
# After training your model
torch.save(model, "deepfake_detector_full.pth")

In [31]:
import os
import cv2
import csv
import torch
import numpy as np
from facenet_pytorch import MTCNN
from torchvision import transforms
from torch.nn.functional import interpolate
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=224, margin=0, device=device)
NUM_FRAMES_PER_VIDEO = 5

def prepare_fft(tensor):
    gray = tensor.mean(dim=0, keepdim=True)
    fft = torch.fft.fft2(gray)
    fft_mag = torch.abs(fft)
    fft_norm = (fft_mag - fft_mag.min()) / (fft_mag.max() - fft_mag.min() + 1e-8)
    return fft_norm.expand(3, -1, -1)

def prepare_motion(curr, prev):
    return torch.abs(curr - prev) if prev is not None else torch.zeros_like(curr)

def predict_on_dataset(model, dataset_path, output_csv="predictions.csv"):
    model.eval()
    video_extensions = (".mp4", ".avi", ".mov")

    with open(output_csv, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["video_path", "label", "predicted_label", "confidence"])

        for label_str in ["real", "fake"]:
            label = 0 if label_str == "real" else 1
            folder_path = os.path.join(dataset_path, label_str)

            for video_file in tqdm(os.listdir(folder_path), desc=f"Processing {label_str}"):
                if not video_file.lower().endswith(video_extensions):
                    continue

                video_path = os.path.join(folder_path, video_file)
                cap = cv2.VideoCapture(video_path)
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                frame_idxs = np.linspace(0, total_frames - 1, NUM_FRAMES_PER_VIDEO, dtype=int)

                rgb_tensors, fft_tensors, motion_tensors = [], [], []
                prev_tensor = None

                for idx in frame_idxs:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                    success, frame = cap.read()
                    if not success:
                        continue

                    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    face = mtcnn(rgb)

                    if face is None:
                        continue

                    face = interpolate(face.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
                    rgb_tensor = face.to(device)

                    fft_tensor = prepare_fft(rgb_tensor)
                    motion_tensor = prepare_motion(rgb_tensor, prev_tensor)
                    prev_tensor = rgb_tensor.clone()

                    rgb_tensors.append(rgb_tensor)
                    fft_tensors.append(fft_tensor)
                    motion_tensors.append(motion_tensor)

                cap.release()

                if not rgb_tensors:
                    writer.writerow([f"{label_str}/{video_file}", label, "No Face Detected", "N/A"])
                    continue

                rgb_batch = torch.stack(rgb_tensors)
                fft_batch = torch.stack(fft_tensors)
                motion_batch = torch.stack(motion_tensors)

                with torch.no_grad():
                    output = model(rgb_batch, fft_batch, motion_batch)
                    pred = torch.sigmoid(output).cpu().numpy().mean()
                    predicted_label = 1 if pred > 0.5 else 0
                    writer.writerow([f"{label_str}/{video_file}", label, predicted_label, round(pred, 4)])


In [42]:
# Example test data paths
REAL_PATH = "/kaggle/input/dataset/DSF/real"
FAKE_PATH = "/kaggle/input/dataset/DSF/fake"

# Select only 3 real and 3 fake videos
real_video_files = sorted(os.listdir(REAL_PATH))[:3]
fake_video_files = sorted(os.listdir(FAKE_PATH))[:3]


In [43]:
NUM_FRAMES_PER_VIDEO = 5  # or your preferred number

def extract_faces_from_list(video_dir, video_files, label, num_frames=5):
    face_list = []
    label_list = []
    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_idxs = np.linspace(0, total_frames-1, num_frames, dtype=int)
        for idx in frame_idxs:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            success, frame = cap.read()
            if not success:
                continue
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face = mtcnn(rgb)
            if face is not None:
                face_list.append(face)
                label_list.append(label)
        cap.release()
    return face_list, label_list

real_faces, real_labels = extract_faces_from_list(REAL_PATH, real_video_files, 0, NUM_FRAMES_PER_VIDEO)
fake_faces, fake_labels = extract_faces_from_list(FAKE_PATH, fake_video_files, 1, NUM_FRAMES_PER_VIDEO)


In [41]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

model.eval()
all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for rgb, fft, motion, labels in test_loader:
        rgb, fft, motion = rgb.to(device), fft.to(device), motion.to(device)
        outputs = model(rgb, fft, motion).squeeze().cpu()
        probs = outputs.numpy()
        preds = (outputs > 0.5).long()
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())
        all_probs.extend(probs.tolist())

# Print metrics
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Real", "Fake"]))
print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
print("ROC AUC Score:", roc_auc_score(all_labels, all_probs))



Classification Report:
              precision    recall  f1-score   support

        Real       0.50      0.93      0.65        15
        Fake       0.50      0.07      0.12        15

    accuracy                           0.50        30
   macro avg       0.50      0.50      0.38        30
weighted avg       0.50      0.50      0.38        30

Confusion Matrix:
 [[14  1]
 [14  1]]
ROC AUC Score: 0.5733333333333333


In [48]:
import torch.nn.functional as F

X_faces = real_faces + fake_faces
y_labels = real_labels + fake_labels

# Resize faces to 224x224
X_faces_tensor = [
    F.interpolate(face.float().unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
    for face in X_faces
]

# Prepare FFT tensors
fft_tensors = [prepare_fft(face) for face in X_faces_tensor]

# Prepare motion tensors
motion_tensors = [
    prepare_motion(X_faces_tensor[i], X_faces_tensor[i-1]) if i > 0 else torch.zeros_like(X_faces_tensor[i])
    for i in range(len(X_faces_tensor))
]


In [49]:
class MultiStreamDeepfakeDataset(Dataset):
    def __init__(self, rgb_faces, fft_images, motion_images, labels, transform=None):
        self.rgb_faces = rgb_faces
        self.fft_images = fft_images
        self.motion_images = motion_images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.rgb_faces)

    def __getitem__(self, idx):
        rgb = self.rgb_faces[idx]
        fft = self.fft_images[idx]
        motion = self.motion_images[idx]
        label = self.labels[idx]
        if self.transform:
            rgb = self.transform(rgb)
            fft = self.transform(fft)
            motion = self.transform(motion)
        return rgb, fft, motion, torch.tensor(label, dtype=torch.float32)

test_dataset = MultiStreamDeepfakeDataset(X_faces_tensor, fft_tensors, motion_tensors, y_labels)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [57]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

model.eval()
all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for rgb, fft, motion, labels in test_loader:
        rgb, fft, motion = rgb.to(device), fft.to(device), motion.to(device)
        outputs = model(rgb, fft, motion).squeeze().cpu()
        probs = outputs.numpy()
        preds = (outputs > 0.5).long()
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())
        all_probs.extend(probs.tolist())

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Real", "Fake"]))
print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
print("ROC AUC Score:", roc_auc_score(all_labels, all_probs))



Classification Report:
              precision    recall  f1-score   support

        Real       0.50      0.93      0.65        15
        Fake       0.50      0.07      0.12        15

    accuracy                           0.50        30
   macro avg       0.50      0.50      0.38        30
weighted avg       0.50      0.50      0.38        30

Confusion Matrix:
 [[14  1]
 [14  1]]
ROC AUC Score: 0.5333333333333333


In [59]:

# Constants
CUSTOM_REAL_PATH = "/kaggle/input/faceforensics/FF++/real"
CUSTOM_FAKE_PATH = "/kaggle/input/faceforensics/FF++/fake"
NUM_FRAMES = 5
mtcnn = MTCNN(image_size=224, margin=0, device=device)

# Face extraction function
def extract_faces_from_videos(video_dir, label, num_frames=5):
    face_list = []
    label_list = []
    
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_idxs = np.linspace(0, total_frames-1, num_frames, dtype=int)

        for idx in frame_idxs:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            success, frame = cap.read()
            if not success: continue

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face = mtcnn(rgb)
            if face is not None:
                face_list.append(face)
                label_list.append(label)
        cap.release()
    
    return face_list, label_list

# Load custom dataset
real_faces, real_labels = extract_faces_from_videos(CUSTOM_REAL_PATH, 0, NUM_FRAMES)
fake_faces, fake_labels = extract_faces_from_videos(CUSTOM_FAKE_PATH, 1, NUM_FRAMES)

all_faces = real_faces + fake_faces
all_labels = real_labels + fake_labels

# Prepare tensors
def prepare_fft(tensor):
    gray = tensor.mean(dim=0, keepdim=True)
    fft = torch.fft.fft2(gray)
    fft_mag = torch.abs(fft)
    fft_norm = (fft_mag - fft_mag.min()) / (fft_mag.max() - fft_mag.min() + 1e-8)
    return fft_norm.expand(3, -1, -1)

def prepare_motion(current, previous):
    return torch.abs(current - previous)

face_tensors = [
    torch.nn.functional.interpolate(f.unsqueeze(0).float(), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
    for f in all_faces
]

fft_tensors = [prepare_fft(face) for face in face_tensors]
motion_tensors = [
    prepare_motion(face_tensors[i], face_tensors[i - 1]) if i > 0 else torch.zeros_like(face_tensors[i])
    for i in range(len(face_tensors))
]

# Inference
all_preds = []
all_probs = []

with torch.no_grad():
    for i in range(len(face_tensors)):
        rgb = face_tensors[i].unsqueeze(0).to(device)
        fft = fft_tensors[i].unsqueeze(0).to(device)
        motion = motion_tensors[i].unsqueeze(0).to(device)

        output = model(rgb, fft, motion).item()
        pred = int(output > 0.5)
        
        all_preds.append(pred)
        all_probs.append(output)

# Evaluation
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["Real", "Fake"]))

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

print("\nROC AUC Score:")
print(roc_auc_score(all_labels, all_probs))


Classification Report:
              precision    recall  f1-score   support

        Real       0.94      0.97      0.95       925
        Fake       0.97      0.94      0.95       985

    accuracy                           0.95      1910
   macro avg       0.95      0.95      0.95      1910
weighted avg       0.95      0.95      0.95      1910


Confusion Matrix:
[[893  32]
 [ 59 926]]

ROC AUC Score:
0.9863174646727947
